# Repeated Holdout Test

**Objectives**
- Load the Model(s)
- Run predictions on the model using the Training/Validation & Holdout/Test* data split using Stratified K-Folds CV (params: k=5 & random_seed=1)
- Record the performance of the models (e.g., accuracy, precision, recall, ROC-AUC, etc.) on every CV instance for use in analysis to determine possible occurence of overfitting.

*The addition of the use of Holdout/Test split here acts as an addition to the original plan of only using Training/Validation data splits.*

**Essentially, this answers the question:** `Is the model overfitted from the training data?`

In [1]:
import pandas as pd

import lightgbm as lgbm
import catboost as catb
 
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import classification_report

from joblib import load

import warnings
warnings.filterwarnings("ignore")

In [2]:
DF_LGBM_TB_TRAIN = pd.read_csv('../Dataset/TB/LGBM_TB.csv', low_memory=False) #<== Point these to the appropriate Training+Validation datasets.
DF_LGBM_IB_TRAIN = pd.read_csv('../Dataset/IB/LGBM_IB.csv', low_memory=False)
DF_LGBM_TB_TEST = pd.read_csv('../Dataset/TB/LGBM_TB_Test.csv', low_memory=False)
DF_LGBM_IB_TEST = pd.read_csv('../Dataset/IB/LGBM_IB_Test.csv', low_memory=False)
display(DF_LGBM_TB_TRAIN.head())
display(DF_LGBM_IB_TRAIN.head())
display(DF_LGBM_TB_TEST.head())
display(DF_LGBM_IB_TEST.head())

DF_CATB_TB_TRAIN = pd.read_csv('../Dataset/TB/CATB_TB.csv', low_memory=False)
DF_CATB_IB_TRAIN = pd.read_csv('../Dataset/IB/CATB_IB.csv', low_memory=False)
DF_CATB_TB_TEST = pd.read_csv('../Dataset/TB/CATB_TB_Test.csv', low_memory=False)
DF_CATB_IB_TEST = pd.read_csv('../Dataset/IB/CATB_IB_Test.csv', low_memory=False)

DF_CATB_IB_TRAIN.iloc[:,1:101] = DF_CATB_IB_TRAIN.iloc[:,1:101].astype('str')
DF_CATB_IB_TRAIN.replace("nan", "NaN", inplace=True)
DF_CATB_IB_TEST.iloc[:,1:101] = DF_CATB_IB_TEST.iloc[:,1:101].astype('str')
DF_CATB_IB_TEST.replace("nan", "NaN", inplace=True)

display(DF_CATB_TB_TRAIN.head())
display(DF_CATB_IB_TRAIN.head())
display(DF_CATB_TB_TEST.head())
display(DF_CATB_IB_TEST.head())

,malware,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,...,t_90,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99
0,1,198,172,117,275,208,286,208,187,208,...,215,198,228,297,2,140,81,8,199,264
1,1,82,240,117,240,117,240,117,240,117,...,209,260,40,209,260,141,260,141,260,141
2,1,82,240,117,240,117,240,117,240,117,...,141,260,141,260,141,260,141,260,141,260
3,1,82,240,117,240,117,240,117,16,228,...,230,240,117,225,89,208,215,192,89,133
4,1,82,240,117,240,117,240,117,240,117,...,159,224,82,260,141,260,141,260,141,260


,malware,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,...,t_90,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99
0,1,198,172,117,275,208,286,187,240,71,...,307,307,307,307,307,307,307,307,307,307
1,1,82,240,117,172,16,11,274,158,215,...,307,307,307,307,307,307,307,307,307,307
2,1,82,240,117,172,16,11,274,158,215,...,307,307,307,307,307,307,307,307,307,307
3,1,82,240,117,16,228,208,71,56,172,...,307,307,307,307,307,307,307,307,307,307
4,1,82,240,117,172,16,11,274,158,215,...,307,307,307,307,307,307,307,307,307,307


,malware,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,...,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99,hash,type
0,1,240,117,240,117,240,117,240,117,240,...,240,117,240,117,172,60,225,35,0fe987c56cfb02db5d810534d6098d93,trojan
1,1,82,198,86,82,274,37,240,117,260,...,274,215,274,158,215,37,158,215,f58d31adac5b879b50ce07a9da086736,trojan
2,1,215,208,228,117,228,240,117,228,159,...,230,35,240,117,208,89,225,35,f07d9fa9d2852bd4b7b36f39dd531b4a,pua
3,1,159,208,260,141,65,208,20,34,215,...,187,135,171,262,208,262,187,262,7a493fa07f0f7d0c9e372eedae03036b,ransomware
4,1,82,240,117,240,117,240,117,240,117,...,260,141,260,141,260,141,260,141,ba60236d9f9fe6cd0a10ffbbf2296669,trojan


,malware,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,...,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99,hash,type
0,1,240,117,228,215,274,158,172,198,208,...,307,307,307,307,307,307,307,307,0fe987c56cfb02db5d810534d6098d93,trojan
1,1,82,198,86,274,37,240,117,260,40,...,307,307,307,307,307,307,307,307,f58d31adac5b879b50ce07a9da086736,trojan
2,1,215,208,228,117,240,159,187,260,141,...,307,307,307,307,307,307,307,307,f07d9fa9d2852bd4b7b36f39dd531b4a,pua
3,1,159,208,260,141,65,20,34,215,172,...,307,307,307,307,307,307,307,307,7a493fa07f0f7d0c9e372eedae03036b,ransomware
4,1,82,240,117,172,16,11,274,158,215,...,307,307,307,307,307,307,307,307,ba60236d9f9fe6cd0a10ffbbf2296669,trojan


,malware,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,...,t_90,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99
0,1,GetSystemInfo,LdrGetDllHandle,LdrGetProcedureAddress,GetSystemDirectoryW,NtAllocateVirtualMemory,SetErrorMode,NtAllocateVirtualMemory,NtFreeVirtualMemory,NtAllocateVirtualMemory,...,NtClose,GetSystemInfo,NtProtectVirtualMemory,NtCreateFile,FindResourceW,SizeofResource,LoadResource,GetFileSize,SetFilePointer,NtReadFile
1,1,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,...,RegEnumKeyExW,RegOpenKeyExW,RegQueryInfoKeyW,RegEnumKeyExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW
2,1,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,...,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW
3,1,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,SetUnhandledExceptionFilter,NtProtectVirtualMemory,...,GetUserNameW,LdrLoadDll,LdrGetProcedureAddress,DrawTextExW,NtDuplicateObject,NtAllocateVirtualMemory,NtClose,NtCreateThreadEx,NtDuplicateObject,NtResumeThread
4,1,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,...,NtDelayExecution,EnumWindows,GetSystemTimeAsFileTime,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW


,malware,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,...,t_90,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99
0,1,GetSystemInfo,LdrGetDllHandle,LdrGetProcedureAddress,GetSystemDirectoryW,NtAllocateVirtualMemory,SetErrorMode,NtFreeVirtualMemory,LdrLoadDll,GetSystemWindowsDirectoryW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,LdrGetDllHandle,SetUnhandledExceptionFilter,CryptAcquireContextW,NtOpenKey,NtQueryValueKey,NtClose,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,LdrGetDllHandle,SetUnhandledExceptionFilter,CryptAcquireContextW,NtOpenKey,NtQueryValueKey,NtClose,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,SetUnhandledExceptionFilter,NtProtectVirtualMemory,NtAllocateVirtualMemory,GetSystemWindowsDirectoryW,CreateActCtxW,LdrGetDllHandle,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,LdrGetDllHandle,SetUnhandledExceptionFilter,CryptAcquireContextW,NtOpenKey,NtQueryValueKey,NtClose,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,malware,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,...,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99,hash,type
0,1,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,...,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrGetDllHandle,FindResourceExW,DrawTextExW,GetSystemMetrics,0fe987c56cfb02db5d810534d6098d93,trojan
1,1,GetSystemTimeAsFileTime,GetSystemInfo,NtCreateMutant,GetSystemTimeAsFileTime,NtOpenKey,NtOpenKeyEx,LdrLoadDll,LdrGetProcedureAddress,RegOpenKeyExW,...,NtOpenKey,NtClose,NtOpenKey,NtQueryValueKey,NtClose,NtOpenKeyEx,NtQueryValueKey,NtClose,f58d31adac5b879b50ce07a9da086736,trojan
2,1,NtClose,NtAllocateVirtualMemory,NtProtectVirtualMemory,LdrGetProcedureAddress,NtProtectVirtualMemory,LdrLoadDll,LdrGetProcedureAddress,NtProtectVirtualMemory,NtDelayExecution,...,GetUserNameW,GetSystemMetrics,LdrLoadDll,LdrGetProcedureAddress,NtAllocateVirtualMemory,NtDuplicateObject,DrawTextExW,GetSystemMetrics,f07d9fa9d2852bd4b7b36f39dd531b4a,pua
3,1,NtDelayExecution,NtAllocateVirtualMemory,RegOpenKeyExW,RegQueryValueExW,RegCloseKey,NtAllocateVirtualMemory,NtOpenFile,NtQueryInformationFile,NtClose,...,NtFreeVirtualMemory,NtCreateSection,NtMapViewOfSection,NtQuerySystemInformation,NtAllocateVirtualMemory,NtQuerySystemInformation,NtFreeVirtualMemory,NtQuerySystemInformation,7a493fa07f0f7d0c9e372eedae03036b,ransomware
4,1,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,...,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,ba60236d9f9fe6cd0a10ffbbf2296669,trojan


,malware,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,...,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99,hash,type
0,1,LdrLoadDll,LdrGetProcedureAddress,NtProtectVirtualMemory,NtClose,NtOpenKey,NtQueryValueKey,LdrGetDllHandle,GetSystemInfo,NtAllocateVirtualMemory,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0fe987c56cfb02db5d810534d6098d93,trojan
1,1,GetSystemTimeAsFileTime,GetSystemInfo,NtCreateMutant,NtOpenKey,NtOpenKeyEx,LdrLoadDll,LdrGetProcedureAddress,RegOpenKeyExW,RegQueryInfoKeyW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f58d31adac5b879b50ce07a9da086736,trojan
2,1,NtClose,NtAllocateVirtualMemory,NtProtectVirtualMemory,LdrGetProcedureAddress,LdrLoadDll,NtDelayExecution,NtFreeVirtualMemory,RegOpenKeyExW,RegQueryValueExW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f07d9fa9d2852bd4b7b36f39dd531b4a,pua
3,1,NtDelayExecution,NtAllocateVirtualMemory,RegOpenKeyExW,RegQueryValueExW,RegCloseKey,NtOpenFile,NtQueryInformationFile,NtClose,LdrGetDllHandle,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7a493fa07f0f7d0c9e372eedae03036b,ransomware
4,1,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,LdrGetDllHandle,SetUnhandledExceptionFilter,CryptAcquireContextW,NtOpenKey,NtQueryValueKey,NtClose,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ba60236d9f9fe6cd0a10ffbbf2296669,trojan


### 0. Defining a helper function for evaluation

In [3]:
import copy
import json

RANDOM_SEED = 1

def get_indexes():
    indexes = []
    for i in range(100):
        indexes.append(f"t_{i}")
    return indexes

def generate_model(model_id, tuned):
    model_holdout = None
    if model_id == 0:
        if tuned == 0:
            return lgbm.LGBMClassifier(random_state=RANDOM_SEED, n_jobs=0, verbose=-1)
        elif tuned == 1: #tuned tb
            return lgbm.LGBMClassifier(random_state=RANDOM_SEED, n_jobs=0, verbose=-1,
                                       boosting_type="gbdt", class_weight="balanced", objective="binary", 
                                       data_sample_strategy="goss", enable_bundle=True, max_depth=5, learning_rate=0.1, 
                                       n_estimators=500, cat_l2=10, num_leaves=16, tree_learner="feature", device="gpu", 
                                       boost_from_average=True, gpu_use_dp=False, max_bin=255)
        elif tuned == 2: #tuned ib
            return lgbm.LGBMClassifier(random_state=RANDOM_SEED, n_jobs=0, verbose=-1,
                                       boosting_type="gbdt", class_weight="balanced", objective="binary", 
                                       data_sample_strategy="goss", enable_bundle=True, max_depth=5, learning_rate=0.1, 
                                       n_estimators=1000, cat_l2=5, num_leaves=32, tree_learner="feature", device="gpu", 
                                       boost_from_average=True, gpu_use_dp=False, max_bin=255)
    elif model_id == 1:
        if tuned==0: #not tuned
            return catb.CatBoostClassifier(random_state=RANDOM_SEED, thread_count=-1, verbose=0, cat_features=get_indexes(), 
                                           nan_mode='Min', one_hot_max_size=256)
        elif tuned==1: #tuned tb 
            return catb.CatBoostClassifier(random_state=RANDOM_SEED, thread_count=-1, verbose=0, cat_features=get_indexes(), 
                                           nan_mode='Min', one_hot_max_size=256,
                                           grow_policy="SymmetricTree", objective="Logloss", bootstrap_type="Bayesian", 
                                           boosting_type="Ordered", max_depth=10, learning_rate=0.1, n_estimators=500, 
                                           l2_leaf_reg=1, task_type="GPU", gpu_cat_features_storage="CpuPinnedMemory", 
                                           train_dir=None, save_snapshot=None)
        elif tuned==2: #tuned ib
            return catb.CatBoostClassifier(random_state=RANDOM_SEED, thread_count=-1, verbose=0, cat_features=get_indexes(), 
                                           nan_mode='Min', one_hot_max_size=256,
                                           grow_policy="SymmetricTree", objective="Logloss", bootstrap_type="Bayesian", 
                                           boosting_type="Ordered", max_depth=10, learning_rate=0.2, n_estimators=100, 
                                           l2_leaf_reg=1, task_type="GPU", gpu_cat_features_storage="CpuPinnedMemory", 
                                           train_dir=None, save_snapshot=None)
    else:
        return None

def stratified_kfolds_eval(train_set, test_set, model:list, model_id:int, tuned:int=0):
    features = train_set.drop('malware', axis=1)
    labels = train_set['malware']
    kfolds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
    print('HOLDOUT FOLD PREDICTIONS')
    print('-------------------------------------------------------')
    ctr = 1
    for train, holdout in kfolds.split(features, labels):
        X_train = features.iloc[train]
        Y_train = labels.iloc[train]
        model_holdout = generate_model(model_id, tuned)
        if model_id == 0:
            model_holdout.fit(X_train, Y_train, categorical_feature=get_indexes())
        elif model_id == 1:
            model_holdout.fit(X_train, Y_train, cat_features=get_indexes())
        else:
            break
        X_holdout = features.iloc[holdout]
        Y_holdout = labels.iloc[holdout]
        Y_pred = model_holdout.predict(X_holdout)
        print(f"Fold: {ctr}")
        print(classification_report(Y_holdout, Y_pred, digits=4))
        print('-------------------------------------------------------')
        ctr += 1
    
    #Comparing each holdout fold result with test split result
    print('\nTEST SPLIT PREDICTIONS')
    print('-------------------------------------------------------')
    #model_holdout.fit(X=features, y=labels)
    Y_pred = model_holdout.predict(test_set.iloc[:,1:101])
    print(classification_report(test_set.iloc[:,0], Y_pred, digits=4))
    print('-------------------------------------------------------')
    
    print('\nTEST SPLIT PREDICTIONS (WITH ACTUAL FILE TRAINED@70:30)') #Re-enactment of what the model file was trained from
    print('-------------------------------------------------------')
    Y_pred = model[0].predict(test_set.iloc[:,1:101])
    print(classification_report(test_set.iloc[:,0], Y_pred, digits=4))
    print('-------------------------------------------------------')
    
#     print('\nTEST SPLIT PREDICTIONS (WITH ACTUAL FILE TRAINED@100:0)') #Re-enactment of what the model file was trained from
#     print('-------------------------------------------------------')
#     Y_pred = model[1].predict(test_set.iloc[:,1:101])
#     print(classification_report(test_set.iloc[:,0], Y_pred, digits=4))
#     print('-------------------------------------------------------')

### 1. LightGBM

In [4]:
default_tb = load('../GBDT_Training/Outputs/LGBM/Default/RYZEN3b_LGBM_TB.model') # <== Point these to the respective .model files
default_ib = load('../GBDT_Training/Outputs/LGBM/Default/RYZEN3b_LGBM_IB.model')
tuned_tb = load('../GBDT_Training/Outputs/LGBM/Tuned/TUNED_RYZEN3b_LGBM_TB.model')
tuned_ib = load('../GBDT_Training/Outputs/LGBM/Tuned/TUNED_RYZEN3b_LGBM_IB.model')

# default_tb_full = load('../GBDT_Training/Outputs/LGBM/Default/RYZEN3c_LGBM_TB.model') # <== Point these to the respective .model files
# default_ib_full = load('../GBDT_Training/Outputs/LGBM/Default/RYZEN3c_LGBM_IB.model')
# tuned_tb_full = load('../GBDT_Training/Outputs/LGBM/Tuned/TUNED_RYZEN3c_LGBM_TB.model')
# tuned_ib_full = load('../GBDT_Training/Outputs/LGBM/Tuned/TUNED_RYZEN3c_LGBM_IB.model')

print("1. Default LGBM TB, 'holdout' from train/validation split vs test split\n")
stratified_kfolds_eval(DF_LGBM_TB_TRAIN, DF_LGBM_TB_TEST, [default_tb], 0, 0)

1. Default LGBM TB, 'holdout' from train/validation split vs test split

HOLDOUT FOLD PREDICTIONS
-------------------------------------------------------
Fold: 1
              precision    recall  f1-score   support

           0     0.9987    0.9909    0.9948      7230
           1     0.9909    0.9988    0.9948      7230

    accuracy                         0.9948     14460
   macro avg     0.9948    0.9948    0.9948     14460
weighted avg     0.9948    0.9948    0.9948     14460

-------------------------------------------------------
Fold: 2
              precision    recall  f1-score   support

           0     0.9996    0.9887    0.9941      7229
           1     0.9888    0.9996    0.9942      7230

    accuracy                         0.9941     14459
   macro avg     0.9942    0.9941    0.9941     14459
weighted avg     0.9942    0.9941    0.9941     14459

-------------------------------------------------------
Fold: 3
              precision    recall  f1-score   support

 

In [5]:
print("2. Default LGBM IB, 'holdout' from train/validation split vs test split\n")
stratified_kfolds_eval(DF_LGBM_IB_TRAIN, DF_LGBM_IB_TEST, [default_ib], 0, 0)

2. Default LGBM IB, 'holdout' from train/validation split vs test split

HOLDOUT FOLD PREDICTIONS
-------------------------------------------------------
Fold: 1
              precision    recall  f1-score   support

           0     0.9992    0.9907    0.9949      7230
           1     0.9908    0.9992    0.9950      7230

    accuracy                         0.9950     14460
   macro avg     0.9950    0.9950    0.9950     14460
weighted avg     0.9950    0.9950    0.9950     14460

-------------------------------------------------------
Fold: 2
              precision    recall  f1-score   support

           0     0.9994    0.9895    0.9944      7229
           1     0.9896    0.9994    0.9945      7230

    accuracy                         0.9945     14459
   macro avg     0.9945    0.9945    0.9945     14459
weighted avg     0.9945    0.9945    0.9945     14459

-------------------------------------------------------
Fold: 3
              precision    recall  f1-score   support

 

In [6]:
print("3. Tuned LGBM TB, 'holdout' from train/validation split vs test split\n")
stratified_kfolds_eval(DF_LGBM_TB_TRAIN, DF_LGBM_TB_TEST, [tuned_tb], 0, 1)

3. Tuned LGBM TB, 'holdout' from train/validation split vs test split

HOLDOUT FOLD PREDICTIONS
-------------------------------------------------------
Fold: 1
              precision    recall  f1-score   support

           0     0.9986    0.9916    0.9951      7230
           1     0.9916    0.9986    0.9951      7230

    accuracy                         0.9951     14460
   macro avg     0.9951    0.9951    0.9951     14460
weighted avg     0.9951    0.9951    0.9951     14460

-------------------------------------------------------
Fold: 2
              precision    recall  f1-score   support

           0     0.9992    0.9895    0.9943      7229
           1     0.9896    0.9992    0.9944      7230

    accuracy                         0.9943     14459
   macro avg     0.9944    0.9943    0.9943     14459
weighted avg     0.9944    0.9943    0.9943     14459

-------------------------------------------------------
Fold: 3
              precision    recall  f1-score   support

   

In [7]:
print("4. Tuned LGBM IB, 'holdout' from train/validation split vs test split\n")
stratified_kfolds_eval(DF_LGBM_IB_TRAIN, DF_LGBM_IB_TEST, [tuned_ib], 0, 2)

4. Tuned LGBM IB, 'holdout' from train/validation split vs test split

HOLDOUT FOLD PREDICTIONS
-------------------------------------------------------
Fold: 1
              precision    recall  f1-score   support

           0     0.9987    0.9918    0.9953      7230
           1     0.9919    0.9988    0.9953      7230

    accuracy                         0.9953     14460
   macro avg     0.9953    0.9953    0.9953     14460
weighted avg     0.9953    0.9953    0.9953     14460

-------------------------------------------------------
Fold: 2
              precision    recall  f1-score   support

           0     0.9987    0.9906    0.9947      7229
           1     0.9907    0.9988    0.9947      7230

    accuracy                         0.9947     14459
   macro avg     0.9947    0.9947    0.9947     14459
weighted avg     0.9947    0.9947    0.9947     14459

-------------------------------------------------------
Fold: 3
              precision    recall  f1-score   support

   

### 2. CatBoost

In [8]:
default_tb = catb.CatBoostClassifier()
default_ib = catb.CatBoostClassifier()
tuned_tb = catb.CatBoostClassifier()
tuned_ib = catb.CatBoostClassifier()
default_tb = default_tb.load_model("../GBDT_Training/Outputs/CATB/Default/RYZEN3b_CATB_TB.model", format='json')
default_ib = default_ib.load_model("../GBDT_Training/Outputs/CATB/Default/RYZEN3b_CATB_IB.model", format='json')
tuned_tb = tuned_tb.load_model("../GBDT_Training/Outputs/CATB/Tuned/TUNED_RYZEN3b_CATB_TB.model", format='json')
tuned_ib = tuned_ib.load_model("../GBDT_Training/Outputs/CATB/Tuned/TUNED_RYZEN3b_CATB_IB.model", format='json')

# default_tb_full = catb.CatBoostClassifier()
# default_ib_full = catb.CatBoostClassifier()
# tuned_tb_full = catb.CatBoostClassifier()
# tuned_ib_full = catb.CatBoostClassifier()
# default_tb_full = default_tb_full.load_model("../GBDT_Training/Outputs/CATB/Default/RYZEN3c_CATB_TB.model", format='json')
# default_ib_full = default_ib_full.load_model("../GBDT_Training/Outputs/CATB/Default/RYZEN3c_CATB_IB.model", format='json')
# tuned_tb_full = tuned_tb_full.load_model("../GBDT_Training/Outputs/CATB/Tuned/TUNED_RYZEN3c_CATB_TB.model", format='json')
# tuned_ib_full = tuned_ib_full.load_model("../GBDT_Training/Outputs/CATB/Tuned/TUNED_RYZEN3c_CATB_IB.model", format='json')

print("1. Default CatBoost TB, 'holdout' from train/validation split vs test split\n")
stratified_kfolds_eval(DF_CATB_TB_TRAIN, DF_CATB_TB_TEST, [default_tb], 1, 0)

1. Default CatBoost TB, 'holdout' from train/validation split vs test split

HOLDOUT FOLD PREDICTIONS
-------------------------------------------------------
Fold: 1
              precision    recall  f1-score   support

           0     0.9981    0.9921    0.9951      7230
           1     0.9922    0.9981    0.9951      7230

    accuracy                         0.9951     14460
   macro avg     0.9951    0.9951    0.9951     14460
weighted avg     0.9951    0.9951    0.9951     14460

-------------------------------------------------------
Fold: 2
              precision    recall  f1-score   support

           0     0.9990    0.9898    0.9944      7229
           1     0.9899    0.9990    0.9944      7230

    accuracy                         0.9944     14459
   macro avg     0.9944    0.9944    0.9944     14459
weighted avg     0.9944    0.9944    0.9944     14459

-------------------------------------------------------
Fold: 3
              precision    recall  f1-score   suppor

In [9]:
print("2. Default CatBoost IB, 'holdout' from train/validation split vs test split\n")
stratified_kfolds_eval(DF_CATB_IB_TRAIN, DF_CATB_IB_TEST, [default_ib], 1, 0)

2. Default CatBoost IB, 'holdout' from train/validation split vs test split

HOLDOUT FOLD PREDICTIONS
-------------------------------------------------------
Fold: 1
              precision    recall  f1-score   support

           0     0.9990    0.9927    0.9958      7230
           1     0.9927    0.9990    0.9959      7230

    accuracy                         0.9959     14460
   macro avg     0.9959    0.9959    0.9959     14460
weighted avg     0.9959    0.9959    0.9959     14460

-------------------------------------------------------
Fold: 2
              precision    recall  f1-score   support

           0     0.9985    0.9907    0.9946      7229
           1     0.9908    0.9985    0.9946      7230

    accuracy                         0.9946     14459
   macro avg     0.9946    0.9946    0.9946     14459
weighted avg     0.9946    0.9946    0.9946     14459

-------------------------------------------------------
Fold: 3
              precision    recall  f1-score   suppor

In [10]:
print("3. Tuned CatBoost TB, 'holdout' from train/validation split vs test split\n")
stratified_kfolds_eval(DF_CATB_TB_TRAIN, DF_CATB_TB_TEST, [tuned_tb], 1, 1)

3. Tuned CatBoost TB, 'holdout' from train/validation split vs test split

HOLDOUT FOLD PREDICTIONS
-------------------------------------------------------
Fold: 1
              precision    recall  f1-score   support

           0     0.9978    0.9918    0.9948      7230
           1     0.9919    0.9978    0.9948      7230

    accuracy                         0.9948     14460
   macro avg     0.9948    0.9948    0.9948     14460
weighted avg     0.9948    0.9948    0.9948     14460

-------------------------------------------------------
Fold: 2
              precision    recall  f1-score   support

           0     0.9983    0.9898    0.9940      7229
           1     0.9899    0.9983    0.9941      7230

    accuracy                         0.9941     14459
   macro avg     0.9941    0.9941    0.9941     14459
weighted avg     0.9941    0.9941    0.9941     14459

-------------------------------------------------------
Fold: 3
              precision    recall  f1-score   support


In [11]:
print("4. Tuned CatBoost IB, 'holdout' from train/validation split vs test split\n")
stratified_kfolds_eval(DF_CATB_IB_TRAIN, DF_CATB_IB_TEST, [tuned_ib], 1, 2)

4. Tuned CatBoost IB, 'holdout' from train/validation split vs test split

HOLDOUT FOLD PREDICTIONS
-------------------------------------------------------
Fold: 1
              precision    recall  f1-score   support

           0     0.9985    0.9928    0.9956      7230
           1     0.9928    0.9985    0.9957      7230

    accuracy                         0.9956     14460
   macro avg     0.9957    0.9956    0.9956     14460
weighted avg     0.9957    0.9956    0.9956     14460

-------------------------------------------------------
Fold: 2
              precision    recall  f1-score   support

           0     0.9987    0.9902    0.9944      7229
           1     0.9903    0.9988    0.9945      7230

    accuracy                         0.9945     14459
   macro avg     0.9945    0.9945    0.9945     14459
weighted avg     0.9945    0.9945    0.9945     14459

-------------------------------------------------------
Fold: 3
              precision    recall  f1-score   support
